# Chapter 6. Test Coverage and Parameterized Tests

## 6.1 Module outline

(1) Parameterized tests with `unittest` and `pytest`

(2) Measuring coverage of tests

(3) Using code coverage metrics when adding test cases

## 6.2 Using a custom `assert` to reduce duplication

### 6.2.1 Parameterized tests

(1) Same function or method

(2) Different parameters

### 6.2.2 Example: Tennis scores

(1) Love-All

(2) Fifteen-Love

(3) Fifteen-thirty

(4) Fifteen-Forty

In [ ]:
# SAVE AS tennis.py

score_names = ["Love", "Fifteen", "Thirty", "Forty"]

def tennis_score(player1_points, player2_points):
    if player1_points == player2_points:
        return "{0}-All".format(score_names[player1_points])
    else:
        return "{0}-{1}".format(score_names[player1_points],
                                    score_names[player2_points])

In [ ]:
# SAVE AS test_tennis.py

import unittest

from tennis import tennis_score

class TennisTest(unittest.TestCase):
    
    def test_even_scores_early_game(self):
        self.assert_tennis_score("Love-All", 0, 0)
        self.assert_tennis_score("Fifteen-All", 1, 1)
        self.assert_tennis_score("Thirty-All", 2, 2)
        
    def test_early_games_with_uneven_scores(self):
        self.assert_tennis_score("Love-Fifteen", 0, 1)
        self.assert_tennis_score("Fifteen-Love", 1, 0)
        self.assert_tennis_score("Love-Thirty", 0, 2)
        self.assert_tennis_score("Thirty-Love", 2, 0)
        
    def assert_tennis_score(self, expected_score, player1_points, player2_points):
        self.assertEqual(expected_score, tennis_score(player1_points, player2_points))

```bash
$ python -m unittest -v
test_check_with_too_high_pressure (test_alarm.AlarmTest) ... ok
test_check_with_too_low_pressure (test_alarm.AlarmTest) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.001s

OK
```

## 6.3 Defining parameterized tests with `unittest`

Use a metaprogramming technique `setattr` to dynamically generate test routines `test_xxx()` from some test data.

In [ ]:
# SAVE AS tennis.py

score_names = ["Love", "Fifteen", "Thirty", "Forty"]

def tennis_score(player1_points, player2_points):
    max_points = max(player1_points, player2_points)
    min_points = min(player1_points, player2_points)
    leading_player = 1 if max_points == player1_points else 2
    
    if max_points >= 4:
        if max_points == min_points:
            return "Deuce"
        elif max_points == min_points + 1:
            return "Advantage Player {}".format(leading_player)
        else:
            return "Win for Player {}".format(leading_player)
    elif max_points == 3 and min_points == 3:
        return "Deuce"
    else:
        if player1_points == player2_points:
            return "{0}-All".format(score_names[player1_points])
        else:
            return "{0}-{1}".format(score_names[player1_points],
                                    score_names[player2_points])

In [ ]:
# SAVE AS test_tennis.py

import unittest

from tennis import tennis_score

test_case_data = \
{   "even_scores" :                     [("Love-All", 0, 0), 
                                         ("Fifteen-All", 1, 1),
                                         ("Thirty-All", 2, 2),
                                        ],
    "early_games_with_uneven_scores" :  [("Love-Fifteen", 0, 1),
                                         ("Fifteen-Love", 1, 0),
                                         ("Thirty-Fifteen", 2, 1),
                                         ("Forty-Thirty", 3, 2),
                                        ],
    "endgame_with_uneven_scores":       [("Advantage Player 1", 4, 3),
                                         ("Advantage Player 2", 5, 6),
                                         ("Advantage Player 1", 13, 12), 
                                        ],
    "endgame_with_even_scores":         [("Deuce", 3, 3),
                                         ("Deuce", 4, 4),
                                         ("Deuce", 14, 14),  
                                        ],
    "there_is_a_winner":                [("Win for Player 1", 4, 0),
                                         ("Win for Player 2", 2, 4),
                                         ("Win for Player 1", 6, 4),
                                        ],

}

def tennis_test_template(*args):
    def foo(self):
        self.assert_tennis_score(*args)
    return foo

class TennisTest(unittest.TestCase):

    def assert_tennis_score(self, expected_score, player1_points, player2_points):
        self.assertEqual(expected_score, tennis_score(player1_points, player2_points))


for behaviour, test_cases in test_case_data.items():
    for tennis_test_case_data in test_cases:
        expected_output, player1_score, player2_score = tennis_test_case_data
        test_name = "test_{0}_{1}_{2}".format(behaviour, player1_score, player2_score)
        tennis_test_case = tennis_test_template(*tennis_test_case_data)
        setattr(TennisTest, test_name, tennis_test_case)

```bash
$ python -m unittest -v
test_early_games_with_uneven_scores_0_1 (test_tennis.TennisTest) ... ok
test_early_games_with_uneven_scores_1_0 (test_tennis.TennisTest) ... ok
test_early_games_with_uneven_scores_2_1 (test_tennis.TennisTest) ... ok
test_early_games_with_uneven_scores_3_2 (test_tennis.TennisTest) ... ok
test_endgame_with_even_scores_14_14 (test_tennis.TennisTest) ... ok
test_endgame_with_even_scores_3_3 (test_tennis.TennisTest) ... ok
test_endgame_with_even_scores_4_4 (test_tennis.TennisTest) ... ok
test_endgame_with_uneven_scores_13_12 (test_tennis.TennisTest) ... ok
test_endgame_with_uneven_scores_4_3 (test_tennis.TennisTest) ... ok
test_endgame_with_uneven_scores_5_6 (test_tennis.TennisTest) ... ok
test_even_scores_0_0 (test_tennis.TennisTest) ... ok
test_even_scores_1_1 (test_tennis.TennisTest) ... ok
test_even_scores_2_2 (test_tennis.TennisTest) ... ok
test_there_is_a_winner_2_4 (test_tennis.TennisTest) ... ok
test_there_is_a_winner_4_0 (test_tennis.TennisTest) ... ok
test_there_is_a_winner_6_4 (test_tennis.TennisTest) ... ok

----------------------------------------------------------------------
Ran 16 tests in 0.001s

OK
```

## 6.4 Defining parameterized tests with `pytest`

Use the decorator `@pytest.mark.parameterize`.

In [ ]:
# SAVE AS test_tennis.py

from tennis import tennis_score
import pytest


examples = (("expected_score", "player1_points", "player2_points", "comment"), 
[
("Love-All", 0, 0, "early game, scores equal"),
("Fifteen-All", 1, 1, "early game, scores equal"),
("Thirty-All", 2, 2, "early game, scores equal"),
("Love-Fifteen", 0, 1, "early game, uneven scores"),
("Fifteen-Love", 1, 0, "early game, uneven scores"),
("Thirty-Fifteen", 2, 1, "early game, uneven scores"),
("Forty-Thirty", 3, 2, "early game, uneven scores"),
("Advantage Player 1", 4, 3, "endgame, with uneven scores"),
("Advantage Player 1", 23, 22, "endgame, with uneven scores"),
("Deuce", 3, 3, "endgame, with even scores"),
("Deuce", 4, 4, "endgame, with even scores"),
("Deuce", 14, 14, "endgame, with even scores"),
("Win for Player 1", 4, 0, "endgame, with winner"),
("Win for Player 2", 1, 4, "endgame, with winner"),
("Win for Player 1", 6, 4, "endgame, with winner"),
])
@pytest.mark.parametrize(*examples)
def test_tennis_scores(expected_score, player1_points, player2_points, comment):
    assert expected_score == tennis_score(player1_points, player2_points)

```bash
$ python -m pytest -v
========================================================= test session starts =========================================================
platform linux -- Python 3.6.4, pytest-3.4.2, py-1.5.2, pluggy-0.6.0 -- /home/renwei/anaconda3/envs/ml/bin/python
cachedir: .pytest_cache
rootdir: /home/renwei/repos/github/learning-ml/python/pluralsight-unit-testing-with-python/parameterized_tennis_pytest, inifile:
collected 15 items                                                                                                                    

test_tennis.py::test_tennis_scores[Love-All-0-0-early game, scores equal] PASSED                                                [  6%]
test_tennis.py::test_tennis_scores[Fifteen-All-1-1-early game, scores equal] PASSED                                             [ 13%]
test_tennis.py::test_tennis_scores[Thirty-All-2-2-early game, scores equal] PASSED                                              [ 20%]
test_tennis.py::test_tennis_scores[Love-Fifteen-0-1-early game, uneven scores] PASSED                                           [ 26%]
test_tennis.py::test_tennis_scores[Fifteen-Love-1-0-early game, uneven scores] PASSED                                           [ 33%]
test_tennis.py::test_tennis_scores[Thirty-Fifteen-2-1-early game, uneven scores] PASSED                                         [ 40%]
test_tennis.py::test_tennis_scores[Forty-Thirty-3-2-early game, uneven scores] PASSED                                           [ 46%]
test_tennis.py::test_tennis_scores[Advantage Player 1-4-3-endgame, with uneven scores] PASSED                                   [ 53%]
test_tennis.py::test_tennis_scores[Advantage Player 1-23-22-endgame, with uneven scores] PASSED                                 [ 60%]
test_tennis.py::test_tennis_scores[Deuce-3-3-endgame, with even scores] PASSED                                                  [ 66%]
test_tennis.py::test_tennis_scores[Deuce-4-4-endgame, with even scores] PASSED                                                  [ 73%]
test_tennis.py::test_tennis_scores[Deuce-14-14-endgame, with even scores] PASSED                                                [ 80%]
test_tennis.py::test_tennis_scores[Win for Player 1-4-0-endgame, with winner] PASSED                                            [ 86%]
test_tennis.py::test_tennis_scores[Win for Player 2-1-4-endgame, with winner] PASSED                                            [ 93%]
test_tennis.py::test_tennis_scores[Win for Player 1-6-4-endgame, with winner] PASSED                                            [100%]

====================================================== 15 passed in 0.04 seconds ======================================================
```

## 6.5 Measuring coverage with `pytest-cov`

### 6.5.1 Package installation

```bash
$ pip install coverage pytest-cov
```

### 6.5.2 Measuring coverage

(1) Get a terminal report.

```bash
$ python -m pytest --cov-report term-missing --cov tennis
========================================================= test session starts =========================================================
platform linux -- Python 3.6.4, pytest-3.4.2, py-1.5.2, pluggy-0.6.0
rootdir: /home/renwei/repos/github/learning-ml/python/pluralsight-unit-testing-with-python/coverage_example_pytest, inifile:
plugins: cov-2.5.1
collected 15 items                                                                                                                    

test_tennis.py ...............                                                                                                  [100%]

----------- coverage: platform linux, python 3.6.4-final-0 -----------
Name        Stmts   Miss  Cover   Missing
-----------------------------------------
tennis.py      32      1    97%   29


====================================================== 15 passed in 0.06 seconds ======================================================
```

(2) Get a html report.

```bash
$ python -m pytest --cov-report html --cov tennis
========================================================= test session starts =========================================================
platform linux -- Python 3.6.4, pytest-3.4.2, py-1.5.2, pluggy-0.6.0
rootdir: /home/renwei/repos/github/learning-ml/python/pluralsight-unit-testing-with-python/coverage_example_pytest, inifile:
plugins: cov-2.5.1
collected 15 items                                                                                                                    

test_tennis.py ...............                                                                                                  [100%]

----------- coverage: platform linux, python 3.6.4-final-0 -----------
Coverage HTML written to dir htmlcov


====================================================== 15 passed in 0.06 seconds ======================================================
```

(3) Use `#pragma: no cover` to mark the function that should be ignored for coverage analysis.

Those lines will be shown as excluded in the html report.

In [ ]:
# SAVE AS tennis.py

def tennis_score(p1points, p2points):
    game = TennisGame("Player 1", "Player 2")
    game.p1points = p1points
    game.p2points = p2points
    return game.score()

class TennisGame:

    def __init__(self, player1Name, player2Name):
        self.player1Name = player1Name
        self.player2Name = player2Name
        self.p1points = 0
        self.p2points = 0
    
    def score(self):
        result = ""
        tempScore=0
        if (self.p1points==self.p2points):
            result = {
                0 : "Love-All",
                1 : "Fifteen-All",
                2 : "Thirty-All",
            }.get(self.p1points, "Deuce")
        elif (self.p1points>=4 or self.p2points>=4):
            minusResult = self.p1points-self.p2points
            if (minusResult==1):
                result ="Advantage " + self.player1Name
            elif (minusResult ==-1):
                result ="Advantage " + self.player2Name
            elif (minusResult>=2):
                result = "Win for " + self.player1Name
            else:
                result ="Win for " + self.player2Name
        else:
            for i in range(1,3):
                if (i==1):
                    tempScore = self.p1points
                else:
                    result+="-"
                    tempScore = self.p2points
                result += {
                    0 : "Love",
                    1 : "Fifteen",
                    2 : "Thirty",
                    3 : "Forty",
                }[tempScore]
        return result

        def won_point(self, playerName): # pragma: no cover
            if playerName == self.player1Name:
                self.p1points += 1
            else:
                self.p2points += 1

In [ ]:
# SAVE AS test_tennis.py

from tennis import tennis_score
import pytest


examples = (("expected_score", "player1_points", "player2_points", "comment"), 
[
("Love-All", 0, 0, "early game, scores equal"),
("Fifteen-All", 1, 1, "early game, scores equal"),
("Thirty-All", 2, 2, "early game, scores equal"),
("Love-Fifteen", 0, 1, "early game, uneven scores"),
("Fifteen-Love", 1, 0, "early game, uneven scores"),
("Thirty-Fifteen", 2, 1, "early game, uneven scores"),
("Forty-Thirty", 3, 2, "early game, uneven scores"),
("Advantage Player 1", 4, 3, "endgame, with uneven scores"),
# Add one test case of "Advantage Player 2"
("Advantage Player 2", 4, 5, "endgame, with uneven scores"),
("Advantage Player 1", 23, 22, "endgame, with uneven scores"),
("Deuce", 3, 3, "endgame, with even scores"),
("Deuce", 4, 4, "endgame, with even scores"),
("Deuce", 14, 14, "endgame, with even scores"),
("Win for Player 1", 4, 0, "endgame, with winner"),
("Win for Player 2", 1, 4, "endgame, with winner"),
("Win for Player 1", 6, 4, "endgame, with winner"),
])
@pytest.mark.parametrize(*examples)
def test_tennis_scores(expected_score, player1_points, player2_points, comment):
    assert expected_score == tennis_score(player1_points, player2_points)

```bash
$ python -m pytest --cov-report term-missing --cov tennis
========================================================= test session starts =========================================================
platform linux -- Python 3.6.4, pytest-3.4.2, py-1.5.2, pluggy-0.6.0
rootdir: /home/renwei/repos/github/learning-ml/python/pluralsight-unit-testing-with-python/coverage_example_pytest, inifile:
plugins: cov-2.5.1
collected 16 items                                                                                                                    

test_tennis.py ................                                                                                                 [100%]

----------- coverage: platform linux, python 3.6.4-final-0 -----------
Name        Stmts   Miss  Cover   Missing
-----------------------------------------
tennis.py      32      0   100%


====================================================== 16 passed in 0.07 seconds ======================================================
```

## 6.6 Measuring coverage of `unittest` tests

(1) Two steps for generating the coverage report

```bash
$ python -m unittest
...............
----------------------------------------------------------------------
Ran 15 tests in 0.001s

OK
$ python -m coverage run -m unittest
...............
----------------------------------------------------------------------
Ran 15 tests in 0.001s

OK
```

(2) To see the report in the terminal,

```
$ python -m coverage report
Name             Stmts   Miss  Cover
------------------------------------
tennis.py           36      5    86%
test_tennis.py      17      0   100%
------------------------------------
TOTAL               53      5    91%
```

You may also view the html report in a browser.

```bash
$ python -m coverage html
```

## 6.7 Use coverage data to add tests to legacy code

Refactor an inventory system.

Set the `branch` flag in a configuration file `.coveragerc` to also measure the branch coverage:

```
[run]
branch = True
```

In [ ]:
# SAVE AS gilded_rose.py

# -*- coding: utf-8 -*-

class GildedRose(object):

    def __init__(self, items):
        self.items = items

    def update_quality(self):
        for item in self.items:
            if item.name != "Aged Brie" and item.name != "Backstage passes to a TAFKAL80ETC concert":
                if item.quality > 0:
                    if item.name != "Sulfuras, Hand of Ragnaros":
                        item.quality = item.quality - 1
            else:
                if item.quality < 50:
                    item.quality = item.quality + 1
                    if item.name == "Backstage passes to a TAFKAL80ETC concert":
                        if item.sell_in < 11:
                            if item.quality < 50:
                                item.quality = item.quality + 1
                        if item.sell_in < 6:
                            if item.quality < 50:
                                item.quality = item.quality + 1
            if item.name != "Sulfuras, Hand of Ragnaros":
                item.sell_in = item.sell_in - 1
            if item.sell_in < 0:
                if item.name != "Aged Brie":
                    if item.name != "Backstage passes to a TAFKAL80ETC concert":
                        if item.quality > 0:
                            if item.name != "Sulfuras, Hand of Ragnaros":
                                item.quality = item.quality - 1
                    else:
                        item.quality = item.quality - item.quality
                else:
                    if item.quality < 50:
                        item.quality = item.quality + 1

    
class Item:
    def __init__(self, name, sell_in, quality):
        self.name = name
        self.sell_in = sell_in
        self.quality = quality

    def __repr__(self):
        return "%s, %s, %s" % (self.name, self.sell_in, self.quality)

In [ ]:
# SAVE AS test_gilded_rose.py

# -*- coding: utf-8 -*-
import pytest

from gilded_rose import Item, GildedRose

examples = (("item_name", "initial_quality", "initial_sellin", "updated_quality", "updated_sellin", "comment"),
             (("foo", 0, 0, 0, -1, "typical item"),
             ("foo", 10, 0, 8, -1, "typical item"),
             ("Sulfuras, Hand of Ragnaros", 0, 0, 0, 0, "exceptional item"),
             ("Sulfuras, Hand of Ragnaros", 10, 0, 10, 0, "exceptional item"),
             ("Sulfuras, Hand of Ragnaros", 10, -1, 10, -1, "exceptional item"),
             ("Aged Brie", 0, 0, 2, -1, "brie item"),
             ("Backstage passes to a TAFKAL80ETC concert", 0, 0, 0, -1, "backstage pass item")
           ))
@pytest.mark.parametrize(*examples)
def test_update_quality(item_name, initial_quality, initial_sellin, updated_quality, updated_sellin, comment):
    item = Item(item_name, initial_sellin, initial_quality)
    gilded_rose = GildedRose([item])
    gilded_rose.update_quality()
    assert item.quality == updated_quality
    assert item.sell_in == updated_sellin

```bash
$ python -m pytest --cov-report term-missing --cov gilded_rose
========================================================= test session starts =========================================================
platform linux -- Python 3.6.4, pytest-3.4.2, py-1.5.2, pluggy-0.6.0
rootdir: /home/renwei/repos/github/learning-ml/python/pluralsight-unit-testing-with-python/gilded_rose_example, inifile:
plugins: cov-2.5.1
collected 7 items                                                                                                                     

test_gilded_rose.py .......                                                                                                     [100%]

----------- coverage: platform linux, python 3.6.4-final-0 -----------
Name             Stmts   Miss  Cover   Missing
----------------------------------------------
gilded_rose.py      36      1    97%   46


====================================================== 7 passed in 0.05 seconds =======================================================
$ python -m pytest --cov-report html --cov gilded_rose
========================================================= test session starts =========================================================
platform linux -- Python 3.6.4, pytest-3.4.2, py-1.5.2, pluggy-0.6.0
rootdir: /home/renwei/repos/github/learning-ml/python/pluralsight-unit-testing-with-python/gilded_rose_example, inifile:
plugins: cov-2.5.1
collected 7 items                                                                                                                     

test_gilded_rose.py .......                                                                                                     [100%]

----------- coverage: platform linux, python 3.6.4-final-0 -----------
Coverage HTML written to dir htmlcov


====================================================== 7 passed in 0.05 seconds =======================================================
```

## 6.8 Good and bad uses for coverage metrics

### 6.8.1 Interpreting coverage data

(1) Find missing test cases

(2) Get legacy code under test

(3) Continuous integration - constant measurement

### 6.8.2 Coverage metric to aim for?

It depends.

### 6.8.3 Test quality != Coverage

100% test coverage != no bugs

Besides code coverage,

(1) Code review

(2) Bug reports

(3) Confidence to refactor

(4) Flickering tests

## 6.9 Module review

(1) Parameterized test with `unittest` and `pytest`

(2) Measuring coverage of tests

* to find missing test cases
* to improve regression protection

(3) Interpreting coverage data